<a target="_parent" href="https://colab.research.google.com/github/gretelai/gretel-blueprints/blob/main/docs/notebooks/amazon/navigator_tabular_amazon_bedrock.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# TODO
# Gretel Navigator Tabular on Azure MaaS 

This Notebook will walk you through deploying Gretel Navigator Tabular as a Bedrock Marketplace Model. You can deploy Gretel Navigator as an endpoint in Bedrock and interact with the model using the Gretel SDK.

This Notebook will walk you through the following steps:

* Deploy Gretel Navigator Tabular on Amazon Bedrock
* Install and configure the Gretel SDK
* Generate synthetic data with the Gretel SDK and the Bedrock Endpoint
* Edit and augment existing data with the Gretel SDK and the Bedrock Endpoint

# TODO
# Deploy Gretel Navigator

To get started, visit the [Amazon Bedrock homepage](https://us-west-2.console.aws.amazon.com/bedrock/home?region=us-west-2#/) in the AWS Console. In this example we'll be using `us-west-2`.

1. Under **Foundation Models**, select **Model Catalog**:

<img src="https://gretel-blueprints-pub.s3.us-west-2.amazonaws.com/navigator_bedrock/1_model-catalog.png" alt="Model Catalog" width="70%">

2. Under **Providers** on the left side, select **Gretel**:

<img src="https://gretel-blueprints-pub.s3.us-west-2.amazonaws.com/navigator_bedrock/2_providers.png" alt="Provider" width="70%">

4. Click on **View subscription options**:

<img src="https://gretel-blueprints-pub.s3.us-west-2.amazonaws.com/navigator_bedrock/3_subscription-options.png" alt="Subscription Options" width="70%">


6. Click on **Subscribe**:

<img src="https://gretel-blueprints-pub.s3.us-west-2.amazonaws.com/navigator_bedrock/4_subscribe.png" alt="Subscribe" width="60%">


8. Wait for the subscription to complete:

<img src="https://gretel-blueprints-pub.s3.us-west-2.amazonaws.com/navigator_bedrock/5_subscription_complete.png" alt="Subscription Complete" width="70%">


10. Once the subscription is complete, click **Deploy**:

<img src="https://gretel-blueprints-pub.s3.us-west-2.amazonaws.com/navigator_bedrock/6_deploy.png" alt="Deploy" width="70%">


12. You should reach a configuration screen like below. For this example, we will use the defaults. Update the fields for your use case and modify the **Advanced Settings** as required.


When you are done with the configuration, click the **Deploy** button on the bottom right.

<img src="https://gretel-blueprints-pub.s3.us-west-2.amazonaws.com/navigator_bedrock/7_config_deploy.png" alt="Configure and Deploy" width="70%">


8. Remain on the page, and you should eventually see something like this:

<img src="https://gretel-blueprints-pub.s3.us-west-2.amazonaws.com/navigator_bedrock/8_in_progress.png" alt="Deployment Progress" width="70%">


Wait for the model to deploy and the **Endpoint status** to change from **Creating** to **In Service**

# Setup

From the **Marketplace deployments** page (see above). Retrieve the **Endpoint Name (ARN)** and set the variable below.

In [1]:
# Install the OpenAI SDK (if you do not already have it)

!pip install -U -qq openai gretel-client

In [3]:
# Import required libraries
from openai import OpenAI
from getpass import getpass

In [4]:
# Set region and get credentials securely
AZURE_ENDPOINT = input("Azure endpoint: ")
AZURE_API_KEY = getpass("Azure API key: ")

# the `AzureOpenAI` client mangles the URL, so we stick with the default `OpenAI` client
oai_client = OpenAI(base_url=AZURE_ENDPOINT, api_key=AZURE_API_KEY)

Azure endpoint:  https://gretelserverlessendpointmaas.eastus2.models.ai.azure.com
Azure API key:  ········


In [5]:
# Create an Azure Adapter using the Gretel SDK

from gretel_client import Gretel

azure_open_ai = Gretel.create_navigator_azure_oai_adapter(oai_client)

# Generate and Augment Datasets using Gretel Navigator

Alright, we're now ready to start creating data! We'll first generate some data using a single prompt, and then we'll add a couple of new columns. Try out some of your own prompts to see how it works.

In [6]:
# First we'll generate some data from only a prompt. We provide a prompt and some existing sample data to guide the generation process.

import pandas as pd

PROMPT = """Generate a mock dataset for users from the Foo company based in France.
Each user should have the following columns:
* first_name: traditional French first names.
* last_name: traditional French surnames.
* email: formatted as the first letter of their first name followed by their last name @foo.io (e.g., jdupont@foo.io)
* gender: Male/Female
* city: a city in France
* country: always 'France'.
"""

table_headers = ["first_name", "last_name", "email", "gender", "city", "country"]
table_data = [
    {
        "first_name": "Lea",
        "last_name": "Martin",
        "email": "lmartin@foo.io",
        "gender": "Female",
        "city": "Lyon",
        "country": "France",
    }
]

SAMPLE_DATA = pd.DataFrame(table_data, columns=table_headers)

metadata, synthetic_df = azure_open_ai.generate(
    "gretelai/auto",
    PROMPT,
    num_records=10,
    sample_data=SAMPLE_DATA,
)

print(synthetic_df)
print("*******")
print(metadata)


Generating data: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:10, 0.96 records/s]

  first_name   last_name               email  gender              city country
0    Antoine        Roux        aroux@foo.io    Male         Marseille  France
1        Léa      Durand      ldurand@foo.io  Female        Strasbourg  France
2    Gaspard    Fournier    gfournier@foo.io    Male          Bordeaux  France
3   Juliette      Pierre      jpierre@foo.io  Female            Rennes  France
4    Étienne    Marchand    emarchand@foo.io    Male          Toulouse  France
5    Aurélie      Benoit      abenoit@foo.io  Female             Lille  France
6     Cédric      Renaud      crenaud@foo.io    Male              Nice  France
7  Charlotte     Garnier     cgarnier@foo.io  Female  Clermont-Ferrand  France
8    Olivier       Dumas       odumas@foo.io    Male            Nantes  France
9      Adèle  Carpentier  acarpentier@foo.io  Female       Montpellier  France
*******
ResponseMetadata(completion_id='b18cae2d-0a46-4b42-b2d8-f347ac1c6225', usage={'completion_tokens': 550, 'prompt_tokens': 99

In [7]:
# Finally, we'll demonstrate Navigator's edit mode, which can augment existing datasets. In this example we'll take our previously
# generated Synthetic DF and ask Navigator to augment it with new columns.

EDIT_PROMPT = """Edit the table and add the following columns:
* occupation: a random occupation
* education level: make it relevant to the occupation
"""

metadata, augmented_df = azure_open_ai.edit(
    "gretelai/auto",
    EDIT_PROMPT,
    seed_data=synthetic_df
)

print(augmented_df)
print("*******")
print(metadata)

Editing data: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:04, 2.05 records/s]

  first_name   last_name               email  gender              city  \
0    Antoine        Roux        aroux@foo.io    Male         Marseille   
1        Léa      Durand      ldurand@foo.io  Female        Strasbourg   
2    Gaspard    Fournier    gfournier@foo.io    Male          Bordeaux   
3   Juliette      Pierre      jpierre@foo.io  Female            Rennes   
4    Étienne    Marchand    emarchand@foo.io    Male          Toulouse   
5    Aurélie      Benoit      abenoit@foo.io  Female             Lille   
6     Cédric      Renaud      crenaud@foo.io    Male              Nice   
7  Charlotte     Garnier     cgarnier@foo.io  Female  Clermont-Ferrand   
8    Olivier       Dumas       odumas@foo.io    Male            Nantes   
9      Adèle  Carpentier  acarpentier@foo.io  Female       Montpellier   

  country  occupation                 education level  
0  France        Chef                   Culinary Arts  
1  France      Lawyer             Juris Doctor (J.D.)  
2  France    Engi